<a href="https://colab.research.google.com/github/jorcisai/APR/blob/master/neuralnets/src/mlp_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MNIST loader

Importing standard packages and tensorflow_datasets to ease data manipulation.

In [18]:
from __future__ import absolute_import, division, print_function
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import os
import numpy as np  
import tensorflow as tf
import tensorflow_datasets as tfds

First, you need to upload mnist_dataset.py and original MNIST data files from your drive

In [3]:
from google.colab import files

uploaded = files.upload()

Saving train-labels-idx1-ubyte.gz to train-labels-idx1-ubyte.gz
Saving train-images-idx3-ubyte.gz to train-images-idx3-ubyte.gz
Saving t10k-labels-idx1-ubyte.gz to t10k-labels-idx1-ubyte.gz
Saving t10k-images-idx3-ubyte.gz to t10k-images-idx3-ubyte.gz


Loading training and test images and labs from array into dataset object. Then, creating pairs of (image,label)

In [52]:
from mnist_dataset import get_mnist

(X, xl), (Y, yl) = get_mnist("./").load_data()
N = X.shape[0]
C = len(set(xl))
#D = X.shape[1]
#mu = np.sum(X,axis=0)/N
#sigma = np.sum((X-mu)**2,axis=0)/N
#X = (X-mu)/(0.9*sigma+0.1*np.ones(D))
#Y = (Y-mu)/(0.9*sigma+0.1*np.ones(D))
#X = np.float32(X)
#Y = np.float32(Y)
Xdat   = tf.data.Dataset.from_tensor_slices(X)
xldat  = tf.data.Dataset.from_tensor_slices(xl)
Xxldat = tf.data.Dataset.zip((Xdat, xldat))
Ydat   = tf.data.Dataset.from_tensor_slices(Y)
yldat  = tf.data.Dataset.from_tensor_slices(yl)
Yyldat = tf.data.Dataset.zip((Ydat, yldat))

Taking a look at a couple of training images and labels after being converted into dataset type.

In [53]:
print(N,C)
for (image,label) in Xxldat.take(1):
  print(label)
  print(image)

60000 10
tf.Tensor(5, shape=(), dtype=int16)
tf.Tensor(
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   3  18  18  18 126 136 175  26 166 255
 247 127   0   0   0   0   0   0   0   0   0   0   0   0  30  36  94 154
 170 253 253 253 253 253 225 172 253 242 195  64   0   0   0   0   0   0
   0   0   0   0   0  49 238 253 253 253 253 253 253 253 253 251  93  82
  82  56  39   0   0   0   0   0   0   0   0   0   0   0   0  18 219

# Experimental design

Use tf.data.Dataset.take and tf.data.Dataset.skip to split training dataset into 90% for training and 10% for dev.

Before being passed into the model, the datasets need to be shuffled and batched. So, first, the complete dataset is shuffled with a fixed seed so that we can repeat the same shuffle of the dataset, then the dataset is split into training, validation and test, and each of these subsets is batched.

In [54]:
Ntr = round(N*0.9)
Ndv = N-Ntr
Nbatch = 1000

Xxldat = Xxldat.shuffle(N,seed=13)

Xxltr = Xxldat.take(Ntr)
Xxltr = Xxltr.batch(Nbatch)
Xxldv = Xxldat.skip(Ntr).take(Ndv)
Xxldv = Xxldv.batch(Nbatch)
Yylte = Yyldat.batch(Nbatch)

# Build the model

Create an empty model and add layers to it.

In [55]:
model = tf.keras.Sequential()

We'll have a series of one or more densely connected layers, with the last one being the output layer. The output layer produces a probability for all the labels. The one with the highest probability is the models prediction of an example's label.

In [56]:
# One or more dense layers.
# Edit the list in the `for` line to experiment with layer sizes.
for units in [800]:
  model.add(tf.keras.layers.Dense(units, activation='relu'))

# Output layer. The first argument is the number of labels.
model.add(tf.keras.layers.Dense(C, activation='softmax'))

Finally, compile the model. For a softmax categorization model, use sparse_categorical_crossentropy as the loss function. You can try other optimizers, but adam is very common.

In [57]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model

This model running for 10 epochs on this data produces decent results

In [58]:
model.fit(Xxltr, epochs=10, validation_data=Xxldv)

Epoch 1/10
54/54 [==============================] - 4s 68ms/step - loss: 15.3120 - accuracy: 0.8397 - val_loss: 2.4931 - val_accuracy: 0.9422
Epoch 2/10
54/54 [==============================] - 4s 68ms/step - loss: 2.0492 - accuracy: 0.9441 - val_loss: 1.1789 - val_accuracy: 0.9575
Epoch 3/10
54/54 [==============================] - 4s 68ms/step - loss: 0.9775 - accuracy: 0.9592 - val_loss: 0.6408 - val_accuracy: 0.9672
Epoch 4/10
54/54 [==============================] - 4s 69ms/step - loss: 0.5528 - accuracy: 0.9701 - val_loss: 0.3048 - val_accuracy: 0.9793
Epoch 5/10
54/54 [==============================] - 4s 68ms/step - loss: 0.3289 - accuracy: 0.9771 - val_loss: 0.2469 - val_accuracy: 0.9828
Epoch 6/10
54/54 [==============================] - 4s 66ms/step - loss: 0.2148 - accuracy: 0.9823 - val_loss: 0.0836 - val_accuracy: 0.9890
Epoch 7/10
54/54 [==============================] - 4s 66ms/step - loss: 0.1311 - accuracy: 0.9870 - val_loss: 0.0786 - val_accuracy: 0.9910
Epoch 8/10
5

# Evaluate the model

Compute accuracy on the test set (accuracy 98.6%)

In [59]:
loss_te, acc_te = model.evaluate(Yylte)

print('\nEval loss: {:.3f}, Eval accuracy: {:.3f}'.format(loss_te, acc_te))

10/10 [==============================] - 0s 28ms/step - loss: 0.7105 - accuracy: 0.9670

Eval loss: 0.710, Eval accuracy: 0.967
